In [1]:
import numpy as np
import random
# import trimesh
from mpl_toolkits import mplot3d
from matplotlib import pyplot
import copy
import os
import matplotlib.patches as mpatches
import sys
import time
import statistics



In [ ]:
def cellStart(state): #bias initial condition
    rand_cell = np.random.randint(0,5)#Random cell type
    if rand_cell == 0: #body centered
        state['CellType'].append('BV')
        state['Volume'].append(2.82)
    elif rand_cell == 1: #body centered verticle
        state['CellType'].append('SC')
        state['Volume'].append(1.82)
    elif rand_cell == 2 or rand_cell == 3 or  rand_cell == 4:  #simple cubic with rounded edges
        state['CellType'].append('FC')
        state['Volume'].append(3.69)
    return state
def cell(state):
    rand_cell = np.random.randint(0,3)#Random cell type
    if rand_cell == 0: #body centered verticle
        state['CellType'].append('BV')
        state['Volume'].append(2.82)
    elif rand_cell == 1: #body centered
        state['CellType'].append('SC')
        state['Volume'].append(1.82)
    elif rand_cell == 2:  #simple cubic with rounded edges
        state['CellType'].append('FC')
        state['Volume'].append(3.69)
    return state

In [ ]:
def constraint(State,SD1,SD2,V_max,A_max): #constraints
    VOL = np.array(State['Volume']).reshape(-1,1)
    CELL =  np.array(State['CellType']).reshape(-1,1)
    POS = np.array(State['Position'])
    cell = CELL.flatten()
    Ly1 = (np.max(SD1[:,1])-np.min(SD1[:,1]))#Length of boundary
    Lx1 = (np.max(SD1[:,0])-np.min(SD1[:,0])) #Length of boundary
    Lz = (np.max(SD1[:,2])-np.min(SD1[:,2]))#Height of boundary
    if len(SD2) != 0: #if multiple support domains
        Lx2 = (np.max(SD2[:,0])-np.min(SD2[:,0])) #Length of boundary
        Ly2 = (np.max(SD2[:,1])-np.min(SD2[:,1]))#Length of boundary
    else:
        A_max_XY2 = 0
        A_max_YZ2 = 0
        Lx2 = 0
        Ly2 = -1
    Area_YZ = 0
    for t in range(len(CELL)): #compute area in contact with side column
        if POS[t,0]+2 == np.max(SD1[:,0]):
            if CELL[t] == 'SC':
                Area_YZ += 2.31
            elif CELL[t] == 'BV':
                Area_YZ += 1.30394
            elif CELL[t] == 'FC':
                Area_YZ += 3.24548
    if len(SD2)!=0: #if multiple support domains
        for t in range(len(CELL)): #compute area in contact with side column
            if POS[t,1] == np.min(SD2[:,1]):
                if CELL[t] == 'SC':
                    Area_YZ += 2.31
                elif CELL[t] == 'BV':
                    Area_YZ += 1.30394
                elif CELL[t] == 'FC':
                    Area_YZ += 3.24548
    Area_XY = 0
    for t in range(len(CELL)):#compute area in contact with top overhang
        if POS[t,2]+2 == Lz:
            if CELL[t] == 'SC':
                Area_XY += 2.31
            elif CELL[t] == 'BV':
                Area_XY += 0.55668
            elif CELL[t] == 'FC':
                Area_XY += 0.86464
    if (Area_XY+Area_YZ)>A_max: #check area constraint
        ver = False
        print('Area Violation:',Area_XY+Area_YZ,'>',A_max)
    else:
        ver = True
        V_sup = sum(VOL)
        if V_sup > V_max:#check volumn constraint
            ver = False
            print('Volume Violation:',V_sup,'>',V_max)

    return ver

In [ ]:
def initialize(SD1,SD2): #create initial state
    h = 2 #height of cell [mm]
    w = 2 #width of cell [mm]
    d = 2 #depth of cell [mm]
    part_num_cellX = int(a/w)
    num_cellX_SD1 = int((np.max(SD1[:,0])-np.min(SD1[:,0]))/w)
    num_cellY_SD1 = int((np.max(SD1[:,1])-np.min(SD1[:,1]))/d)
    if len(SD2) != 0:
        num_cellX_SD2 = int((np.max(SD2[:,0])-np.min(SD2[:,0]))/w)
        num_cellY_SD2 = int((np.max(SD2[:,1])-np.min(SD2[:,1]))/d)
    num_cellZ = int((np.max(SD1[:,2]))/h)
    minXb1 = np.min(SD1[:,0])
    minYb1 = np.min(SD1[:,1])
    minZb1 = np.min(SD1[:,2])
    Coord = np.empty([0,3])
    T_state = {'CellType':[],'Volume':[],'Position':[]}
    for s in range(num_cellZ):
        for k in range(num_cellY_SD1):
            for t in range(num_cellX_SD1):
                NU_cell = {'CellType':[],'Volume':[]}
                NU_cell = cell(NU_cell)
                x = minXb1+t*w
                y = minYb1+k*d            
                z = minZb1+s*h
                col = ([x,y,z])
                T_state = add(T_state,NU_cell,col)
    if len(SD2) != 0:
        minXb2 = np.min(SD2[:,0])
        minYb2 = np.min(SD2[:,1])
        minZb2 = np.min(SD2[:,2])
        for s in range(num_cellZ):
            for k in range(num_cellY_SD2):
                for t in range(num_cellX_SD2):
                    NU_cell = {'CellType':[],'Volume':[]}
                    NU_cell = cell(NU_cell)
                    x = minXb2+t*w
                    y = minYb2+k*d            
                    z = minZb2+s*h
                    col = ([x,y,z])
                    T_state = add(T_state,NU_cell,col)
    return T_state
def add(new_state,new_cell,new_Cent): #add new cell
    new_state['CellType'].append(new_cell['CellType'][0])
    new_state['Volume'].append(new_cell['Volume'][0])
    new_state['Position'].append(new_Cent)
    return new_state

def evaluate(a,b,c,d,c2,X_start,Y_start,State,SD1,SD2,T,init,DisMax,StrMax):   #Evaluate Objective function and constraints
    num_cells=len(State['CellType'])
    VOL = np.array(State['Volume']).reshape(-1,1)
    CELL = np.array(State['CellType']).reshape(-1,1)
    POS = np.array(State['Position'])
    cell = CELL.flatten()
    fin = 0
    h =1*10**-3
    Dis, Stress, E = FEM_Thermal(POS,CELL,cell,a,b,c,d,c2,X_start,Y_start,fin,h,init,DisMax,StrMax)   #Run PyAnsys

    A_max_XY1 = (b*c) #total top overhang area
    A_max_YZ = (c*d) #total side column area
    A_max_XY2 = 0
    A_max_YZ2 = 0
    if c2 != 0:
        A_max_XY2 = (b*c2) #25% of max area fill
        A_max_YZ2 = (d*c2) #25% of max area fill
    else:

        Lx2 = 0
        Ly2 = -1
    
    V_sup = sum(VOL)
    
    Area_YZ = 0
    for t in range(len(CELL)):
        if POS[t,0]+2 == np.max(SD1[:,0]):#Area in contact with side column
            if CELL[t] == 'SC':
                Area_YZ += 2.31
            elif CELL[t] == 'BV':
                Area_YZ += 1.30394
            elif CELL[t] == 'FC':
                Area_YZ += 3.24548
    if c2!=0: #if multiple support domains
        for t in range(len(CELL)): #Area in contact with side column
            if POS[t,1] == np.min(SD2[:,1]):
#                 print(POS[t])
                if CELL[t] == 'SC':
                    Area_YZ += 2.31
                elif CELL[t] == 'BV':
                    Area_YZ += 1.30394
                elif CELL[t] == 'FC':
                    Area_YZ += 3.24548
    Area_XY = 0
    for t in range(len(CELL)): #Area in contact with top overhang
        if POS[t,2]+2 == d:
            if CELL[t] == 'SC':
                Area_XY += 2.31
            elif CELL[t] == 'BV':
                Area_XY += 0.55668
            elif CELL[t] == 'FC':
                Area_XY += 0.86464

    return E,V_sup,Area_XY,Area_YZ,init,Dis,Stress

In [ ]:
def metopolis(E_prev,Vol, Area_XY,Area_YZ,a,b,c,d,c2,new_state,prev_state,T,SD1,SD2,init,Dis,Str,DisMax,StrMax,X_start,Y_start): #boolean of True/False
    E1 = E_prev
    Vol1=Vol
    Area_XY1=Area_XY
    Area_YZ1=Area_YZ
    Dis1 = Dis
    Str1 = Str
    E2,Vol2,Area_XY2,Area_YZ2,init,Dis2,Str2= evaluate(a,b,c,d,c2,X_start,Y_start,new_state,SD1,SD2,T,init,DisMax,StrMax)
        
    if E1 < E2:
#         P_acc = np.exp(-(E1-E2)/T) #maxmize
        P_acc = np.exp(-(E2-E1)/T) #minimize
        comp = np.random.uniform(0, 1)
        if P_acc > comp:
            accept = True
            E = E2
            Vol = Vol2
            Area_XY = Area_XY2
            Area_YZ = Area_YZ2
            Dis = Dis2
            Str = Str2
        else:
            accept = False
            P_acc = 0
            E = E1
            Vol = Vol1
            Area_XY = Area_XY1
            Area_YZ = Area_YZ1
            Dis = Dis1
            Str = Str1
    elif E2 == False:
        accept = False
        P_acc = 0
        E = E1
        Vol = Vol1
        Area_XY = Area_XY1
        Area_YZ = Area_YZ1
        Dis = Dis1
        Str = Str1
    else:
        P_acc = 1
        accept = True
        E = E2
        Vol = Vol2
        Area_XY = Area_XY2
        Area_YZ = Area_YZ2
        Dis = Dis2
        Str = Str2
    return accept,E, Vol, Area_XY,Area_YZ,P_acc,Dis, Str

In [ ]:
def ShapeAnneal(state,T,limit,n,alpha,SD1,SD2,a,b,c,d,c2,E,Vol, Area_XY,Area_YZ,init,V_max,A_max,Dis,Str,DisMax,StrMax,X_start,Y_start):
    T_0 = T #initial Temperature
    P_acc = 1
    Res_loss = [] #initialize objective list for storage at each iteration
    E_loss = []#initialize objective list for storage at each temperature
    V_loss = [] #initialize volume list for storage at each temperature
    TotalArea_loss = [] #initialize total area list for storage at each temperature
    Str_loss = [] #initialize p-Norm stress list for storage at each temperature
    Dis_loss = []
    TotalArea_iter = [] #initialize total area list for storage at each iteration
    V_iter = [] #initialize volume list for storage at each iteration
    Temp = [] #initialize temperature list for storage at each iteration
    P_acc_iter = [] #initialize probability of acceptance list for storage at each iteration
    Str_iter = [] #initialize p-Norm stress list for storage at each iteration
    Dis_iter = [] #initialize displacement list for storage at each iteration
#     Iter = 0
    while T > 0.1:
        success = 0 
        for i in range(n):
            prev_state = copy.deepcopy(state)
            Total=b/2*c/2*d/2+b/2*c2/2*d/2
            Int = T_0/2
            Tun = 1
            if i == 0:
                if T == Int:
                    print('Intermediate Zone')
                elif T< Tun:
                    print('Tuning Zone')
            if T<=T_0 and T > Int:
                RANGE = 17 #number of swaps in exploration stage
            elif T>Tun and T<= Int:
                RANGE = 17 #number of swaps in intermediate stage
            else:
                RANGE = 17 #number of swaps in fine-tuning stage
            new_state = copy.deepcopy(state)
            for I in range (RANGE): #begin swapping
                new_cell = {'CellType':[],'Volume':[]} #initialize new cell
                new_cell = cell(new_cell) #obtain new random cell
                rand_cell_select = np.random.randint(0,len(new_state['CellType'])) #randomly select unit cell
                new_state['CellType'][rand_cell_select]=new_cell['CellType'][0]
                new_state['Volume'][rand_cell_select]=new_cell['Volume'][0]
            ver2 = constraint(new_state,SD1,SD2,V_max,A_max) #check constraints
            if ver2 == True: #constraints satisfied
                test,E_curr,Vol_curr, Area_XY_curr,Area_YZ_curr,P_acc,Dis_curr, Str_curr = metopolis(E,Vol, Area_XY,Area_YZ,a,b,c,d,c2,new_state,prev_state,T,SD1,SD2,init,Dis,Str,DisMax,StrMax,X_start,Y_start)
                E = E_curr
                Vol = Vol_curr
                Area_XY = Area_XY_curr
                Area_YZ = Area_YZ_curr
                Dis = Dis_curr
                Str = Str_curr
            else:
                test = False 
            if test == True:
                state = new_state
                print('New Objective:',E)
                success += 1
            else:
                state = prev_state
#             print('Successes',success)
            if success > limit:
                print('~~~~~~~~New Temp~~~~~~~~~~')
                break
            P_acc_iter.append(P_acc)
            Res_loss.append(E)
            V_iter.append(Vol)
            TotalArea_iter.append(Area_XY+Area_YZ)
            Str_iter.append(Str)
            Dis_iter.append(Dis)
        if success == 0:
#             print('Success = 0')
            break
        T = T*alpha
        E_loss.append(E)
        V_loss.append(Vol)
        TotalArea_loss.append(Area_XY+Area_YZ)
        Str_loss.append(Str)
        Dis_loss.append(Dis)
        Temp.append(T)
#         print('Temperature =',T)
    print('Final Objective',E,'Final Volume',np.round(Vol,3),'Final Area',np.round(Area_XY+Area_YZ,3), 'Final Displacement', np.round(Dis,3), 'Final Stress', np.round(Str,3))
    print('Final State',state)
#     print('Final Temperature =',T)
#     print('Final Reistance Value =',Value)
    return state,Res_loss,E_loss,V_loss,TotalArea_loss,Temp,V_iter,TotalArea_iter,P_acc_iter,Str_iter,Dis_iter


In [ ]:
##### 
start_time = time.time()
T = 50
n = 100

limit = n/2
alpha = 0.5
np.set_printoptions(threshold=sys.maxsize)
CellCount = []
SOLU = []
start = 0
end = 3
noSD = 2
for RUN in range(start,end):
    start_time = time.time()
    print('Temp:',T,'alpha',alpha,'n',n,'limit',limit)
    a = 2
    b = 8
    c = 48
    d = 24
    e = 2
    SD1 = np.array([[0,0,0],[0,c,0],[b,0,0],[b,c,0],[0,0,d],[0,c,d],[b,0,d],[b,c,d]])
    if noSD >1: #if more than 1 support domain
        X_start = 12
        Y_start = 49.5
        c2 = 26
        SD2 = np.array([[X_start,Y_start+a,0],[X_start,Y_start+a+b,0],[X_start+c2,Y_start+a,0],[X_start+c2,Y_start+a+b,0],[X_start,Y_start+a,d],[X_start,Y_start+a+b,d],[X_start+c2,Y_start+a,d],[X_start+c2,Y_start+a+b,d]])
    else:
        c2 = 0
        X_start = 0
        Y_start = 0
        SD2 = []
    state = initialize(SD1,SD2) #create initial state

    %run ./HorizontalSupport_MAPDL.ipynb
    mapdl.exit()
    path = os.getcwd()
    mapdl = launch_mapdl(nproc = 8, run_location=path,override = True, license_type = "ansys",clearnup_on_exit = True)
    mapdl.clear()           
    
    print('state',state)
    V_max = 5900 #maximum volume
    A_max = 1360 #maximum area
    DisMax = 76 #maximum total displacement
    StrMax = 0.66 #maximum p-Norm Stress
    if RUN == 0:
        print('V_max',(b*c*d+b*c2*d))
        print('A_max',((b*c+d*c)+(d*c2+b*c2)))
    init = 0
    E,Vol,Area_XY,Area_YZ,init,Dis,Str= evaluate(a,b,c,d,c2,X_start,Y_start,state,SD1,SD2,T,init,DisMax,StrMax)
    init = 1
    print('Initial Objective',E,'Volume',np.round(Vol,3),'Area',np.round(Area_XY+Area_YZ,3),'Max Displacement',np.round(Dis,3),'Max Stress',np.round(Str,3))

    solutions,R_loss,E_loss,V_loss,TotalArea_loss,Temp,V_iter,TotalArea_iter,P_acc_iter,Str_iter,Dis_iter= ShapeAnneal(state,T,limit,n,alpha,SD1,SD2,a,b,c,d,c2,E,Vol, Area_XY,Area_YZ,init,V_max,A_max,Dis,Str,DisMax,StrMax,X_start,Y_start)
    mapdl.exit()
    CELL = np.array(solutions['CellType'])
    SOLU.append(solutions)
    sc = np.count_nonzero(CELL == 'SC')
    bv = np.count_nonzero(CELL == 'BV')
    fc = np.count_nonzero(CELL == 'FC')
    Cell_count = [sc,bv,fc]
    CellCount.append(Cell_count)
    print('Simple Cubic:', sc,'Body-Centered Cubic:', bv,'Face-Centered Vertile Cubic:',fc)

    print("--- %s seconds ---" % (time.time() - start_time))    
    Iter_data = np.concatenate((np.array(R_loss).reshape(-1,1),np.round(np.array(V_iter).reshape(-1,1),3),np.round(np.array(TotalArea_iter).reshape(-1,1),3),np.round(np.array(Str_iter).reshape(-1,1),3),np.round(np.array(Dis_iter).reshape(-1,1),3)),axis = 1)
    with open('TSA in %i' %RUN + '.txt', 'w') as f:
        content = str(Iter_data)
        f.write(content)
        f.close()
CC = np.array(CellCount)
with open('CellCount_TSA_%i'%start+'-%i'%end+'.txt', 'w') as f:
    content = str(CC)
    f.write(content)
    f.close()
with open('SOLU_TSA_%i'%start+'-%i'%end+'.txt', 'w') as f:
    content = str(SOLU)
    f.write(content)
    f.close()